In [ ]:
import google.generativeai as genai

genai.configure(api_key="api_key")


def get_completion(prompt):
    """
    Get the completion for a given prompt using the specified model.
    Returns the answer with the highest score.
    """
    model = genai.GenerativeModel(
            "models/gemini-2.0-flash",
            system_instruction="You are a user.",
        )
    response = model.generate_content(prompt)
    return response.text


## Reference Text

For the following text:
Needed a nice lamp for my bedroom, and this one had 
additional storage and not too high of a price point. 
Got it fast.  The string to our lamp broke during the 
transit and the company happily sent over a new one. 
Came within a few days as well. It was easy to put 
together.  I had a missing part, so I contacted their 
support and they very quickly got me the missing piece! 
Lumina seems to me to be a great company that cares 
about their customers and products!!

Validate if this author of this text is happy or not (answer in a single word: Positive/Negative)

In [ ]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## Coding

Instruct the AI to write a the best unit test for the following code:
```
import requests

def get_user_data(user_id):
    url = f"https://api.example.com/users/{user_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()
```


In [ ]:
code = """
import requests

def get_user_data(user_id):
    url = f"https://api.example.com/users/{user_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()
"""

prompt = f"""
I have the following code: '''{code}'''. Write a unit test using pytest patch 
Give a solution using a decorator 
"""

print(get_completion(prompt))

In [ ]:
prompt = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

print(get_completion(prompt))

## Files

You have a log file (app.log) that contains various events, such as errors, warnings, and informational messages. The goal is to extract key insights like:
	1.	The total number of errors.
	2.	The most common error message.
	3.	The timestamp of the last error.

In [ ]:
prompt = """
You are provided with a log file containing application events. Each line includes a timestamp, a log level (e.g., INFO, ERROR, WARNING), and a message. Your task is to analyze the file and return only the results for the following:
1. The total number of entries with the log level `ERROR`.
2. The most common error message.
3. The timestamp of the last error in the log.

Only return the results in this exact format:
- Total Errors: <number>
- Most Common Error: "<error message>"
- Last Error Timestamp: <timestamp>

Do not provide any Python code or explanations—just the results.
"""

log_file_path = "app.log"  
log_file = genai.upload_file(path=log_file_path, display_name="Log Output")

model = genai.GenerativeModel(model_name="gemini-1.5-flash")  
response = model.generate_content([log_file, prompt])
print(response.text)

In [ ]:
!pip install pandas --quiet

import pandas as pd
import re
from collections import Counter

# Read the log file content
with open("app.log", "r") as file:
    log_content = file.readlines()

# Step 1: Parse the log content into a structured format
logs = []
for line in log_content:
    match = re.match(r"(?P<timestamp>\S+ \S+) \[(?P<level>\w+)] (?P<message>.+)", line)
    if match:
        logs.append(match.groupdict())

# Step 2: Convert to DataFrame for analysis
df = pd.DataFrame(logs)

# Step 3: Filter error logs
error_logs = df[df['level'] == 'ERROR']

# Step 4: Count total errors
total_errors = len(error_logs)

# Step 5: Find the most common error message
most_common_error = Counter(error_logs['message']).most_common(1)[0][0] if not error_logs.empty else None

# Step 6: Find the timestamp of the last error
last_error_timestamp = error_logs['timestamp'].iloc[-1] if not error_logs.empty else None

# Print the results
print(f"Total Errors: {total_errors}")
print(f"Most Common Error: \"{most_common_error}\"")
print(f"Last Error Timestamp: {last_error_timestamp}")